In [1]:
import scanpy as sc

data_dir = '../../dataset/HiT_CA4_DGC_filtered.h5ad'
adata = sc.read_h5ad(data_dir)
print(adata.X)
sc.pp.normalize_total(adata,target_sum=1e4)
sc.pp.log1p(adata)


<Compressed Sparse Row sparse matrix of dtype 'float32'
	with 190662798 stored elements and shape (56367, 18752)>
  Coords	Values
  (0, 10785)	5.0
  (0, 13830)	3.0
  (0, 9792)	1.0
  (0, 361)	3.0
  (0, 11996)	31.0
  (0, 11375)	1.0
  (0, 11866)	1.0
  (0, 1413)	1.0
  (0, 3893)	5.0
  (0, 13348)	3.0
  (0, 8872)	3.0
  (0, 1239)	2.0
  (0, 17254)	5.0
  (0, 11793)	4.0
  (0, 9121)	1.0
  (0, 1474)	2.0
  (0, 10369)	2.0
  (0, 9327)	1.0
  (0, 11060)	1.0
  (0, 10047)	1.0
  (0, 7166)	1.0
  (0, 7878)	2.0
  (0, 16110)	3.0
  (0, 6831)	1.0
  (0, 9236)	3.0
  :	:
  (56366, 17768)	4.0
  (56366, 8372)	1.0
  (56366, 1045)	2.0
  (56366, 17383)	1.0
  (56366, 1783)	5.0
  (56366, 13757)	3.0
  (56366, 14311)	3.0
  (56366, 4387)	1.0
  (56366, 16352)	4.0
  (56366, 103)	2.0
  (56366, 12074)	1.0
  (56366, 10922)	1.0
  (56366, 8267)	1.0
  (56366, 6187)	2.0
  (56366, 8250)	1.0
  (56366, 17441)	1.0
  (56366, 4186)	1.0
  (56366, 212)	1.0
  (56366, 16186)	1.0
  (56366, 10277)	1.0
  (56366, 9604)	1.0
  (56366, 1530)	3.0
  (5

In [2]:
adata

AnnData object with n_obs × n_vars = 56367 × 18752
    obs: 'roi', 'organism_ontology_term_id', 'disease_ontology_term_id', 'self_reported_ethnicity_ontology_term_id', 'assay_ontology_term_id', 'sex_ontology_term_id', 'development_stage_ontology_term_id', 'donor_id', 'suspension_type', 'dissection', 'fraction_mitochondrial', 'fraction_unspliced', 'cell_cycle_score', 'total_genes', 'total_UMIs', 'sample_id', 'supercluster_term', 'cluster_id', 'subcluster_id', 'cell_type_ontology_term_id', 'tissue_ontology_term_id', 'is_primary_data', 'tissue_type', 'cell_type', 'assay', 'disease', 'organism', 'sex', 'tissue', 'self_reported_ethnicity', 'development_stage', 'observation_joinid'
    var: 'Biotype', 'Chromosome', 'End', 'Gene', 'Start', 'feature_is_filtered', 'feature_name', 'feature_reference', 'feature_biotype', 'feature_length', 'feature_type'
    uns: 'BT_FT_predictions_42', 'BT_FT_target_names_42', 'BT_FT_train_loss_42', 'BT_FT_val_loss_42', 'BT_FT_y_test_42', 'RM_FT_predictions_42', 

In [3]:
%load_ext autoreload
%autoreload 2
import torch
import lightning.pytorch as pl
from self_supervision.models.lightning_modules.cellnet_autoencoder import MLPBarlowTwins
from self_supervision.estimator.cellnet import EstimatorAutoEncoder

# 设置你的 .ckpt 文件路径
ckpt_path = "../../sc_pretrained/Pretrained Models/BarlowTwins.ckpt"

# 模型参数
units_encoder = [512, 512, 256, 256, 64]

# 初始化 EstimatorAutoEncoder 实例
estim = EstimatorAutoEncoder(data_path=None)  # 如果没有实际数据路径，可以设置为None

# 加载预训练模型
estim.model = MLPBarlowTwins(
        gene_dim=19331,  # 根据你的数据调整
        batch_size=2048,  # 根据你的需要调整
        units_encoder=units_encoder,
        CHECKPOINT_PATH=ckpt_path
    )


estim.trainer = pl.Trainer(accelerator="gpu", devices=1 if torch.cuda.is_available() else None)
estim.model

/home/hanchuangyi/miniconda3/envs/ssl/lib/python3.10/site-packages/merlin/dtypes/mappings/tf.py:52: UserWarning: Tensorflow dtype mappings did not load successfully due to an error: No module named 'tensorflow'
  warn(f"Tensorflow dtype mappings did not load successfully due to an error: {exc.msg}")
/home/hanchuangyi/miniconda3/envs/ssl/lib/python3.10/site-packages/merlin/dtypes/mappings/triton.py:53: UserWarning: Triton dtype mappings did not load successfully due to an error: No module named 'tritonclient'
  warn(f"Triton dtype mappings did not load successfully due to an error: {exc.msg}")
GPU available: True (cuda), used: True


TPU available: False, using: 0 TPU cores


HPU available: False, using: 0 HPUs


MLPBarlowTwins(
  (train_metrics): MetricCollection(
    (explained_var_uniform): ExplainedVariance()
    (explained_var_weighted): ExplainedVariance()
    (mse): MeanSquaredError(),
    prefix=train_
  )
  (val_metrics): MetricCollection(
    (explained_var_uniform): ExplainedVariance()
    (explained_var_weighted): ExplainedVariance()
    (mse): MeanSquaredError(),
    prefix=val_
  )
  (test_metrics): MetricCollection(
    (explained_var_uniform): ExplainedVariance()
    (explained_var_weighted): ExplainedVariance()
    (mse): MeanSquaredError(),
    prefix=test_
  )
  (inner_model): MLP(
    (0): Linear(in_features=19331, out_features=512, bias=True)
    (1): SELU()
    (2): Dropout(p=0.0, inplace=False)
    (3): Linear(in_features=512, out_features=512, bias=True)
    (4): SELU()
    (5): Dropout(p=0.0, inplace=False)
    (6): Linear(in_features=512, out_features=256, bias=True)
    (7): SELU()
    (8): Dropout(p=0.0, inplace=False)
    (9): Linear(in_features=256, out_features=25

In [4]:
# 加载 checkpoint
checkpoint = torch.load(ckpt_path)
estim.model.inner_model.load_state_dict({k.replace('backbone.', ''): v for k, v in checkpoint.items() if 'backbone' in k})

# 设定模型为评估模式
estim.model.eval()

MLPBarlowTwins(
  (train_metrics): MetricCollection(
    (explained_var_uniform): ExplainedVariance()
    (explained_var_weighted): ExplainedVariance()
    (mse): MeanSquaredError(),
    prefix=train_
  )
  (val_metrics): MetricCollection(
    (explained_var_uniform): ExplainedVariance()
    (explained_var_weighted): ExplainedVariance()
    (mse): MeanSquaredError(),
    prefix=val_
  )
  (test_metrics): MetricCollection(
    (explained_var_uniform): ExplainedVariance()
    (explained_var_weighted): ExplainedVariance()
    (mse): MeanSquaredError(),
    prefix=test_
  )
  (inner_model): MLP(
    (0): Linear(in_features=19331, out_features=512, bias=True)
    (1): SELU()
    (2): Dropout(p=0.0, inplace=False)
    (3): Linear(in_features=512, out_features=512, bias=True)
    (4): SELU()
    (5): Dropout(p=0.0, inplace=False)
    (6): Linear(in_features=512, out_features=256, bias=True)
    (7): SELU()
    (8): Dropout(p=0.0, inplace=False)
    (9): Linear(in_features=256, out_features=25

In [5]:
import pandas as pd
var_df = pd.read_parquet('../../sc_pretrained/var.parquet')
var_df

,feature_id,feature_name
0,ENSG00000186092,OR4F5
1,ENSG00000284733,OR4F29
2,ENSG00000284662,OR4F16
3,ENSG00000187634,SAMD11
4,ENSG00000188976,NOC2L
...,...,...
19326,ENSG00000288702,UGT1A3
19327,ENSG00000288705,UGT1A5
19328,ENSG00000182484,WASH6P
19329,ENSG00000288622,PDCD6-AHRR


In [6]:
all_genes = var_df['feature_name'].tolist()
all_genes

['OR4F5',
 'OR4F29',
 'OR4F16',
 'SAMD11',
 'NOC2L',
 'KLHL17',
 'PLEKHN1',
 'PERM1',
 'HES4',
 'ISG15',
 'AGRN',
 'RNF223',
 'C1orf159',
 'TTLL10',
 'TNFRSF18',
 'TNFRSF4',
 'SDF4',
 'B3GALT6',
 'C1QTNF12',
 'UBE2J2',
 'SCNN1D',
 'ACAP3',
 'PUSL1',
 'INTS11',
 'CPTP',
 'TAS1R3',
 'DVL1',
 'MXRA8',
 'AURKAIP1',
 'CCNL2',
 'MRPL20',
 'ANKRD65',
 'TMEM88B',
 'VWA1',
 'ATAD3C',
 'ATAD3B',
 'ATAD3A',
 'TMEM240',
 'SSU72',
 'FNDC10',
 'MIB2',
 'MMP23B',
 'CDK11B',
 'SLC35E2B',
 'CDK11A',
 'NADK',
 'GNB1',
 'CALML6',
 'TMEM52',
 'CFAP74',
 'GABRD',
 'PRKCZ',
 'FAAP20',
 'SKI',
 'MORN1',
 'RER1',
 'PEX10',
 'PLCH2',
 'PANK4',
 'HES5',
 'TNFRSF14',
 'PRXL2B',
 'MMEL1',
 'TTC34',
 'ACTRT2',
 'PRDM16',
 'ARHGEF16',
 'MEGF6',
 'TPRG1L',
 'WRAP73',
 'TP73',
 'CCDC27',
 'SMIM1',
 'LRRC47',
 'CEP104',
 'DFFB',
 'C1orf174',
 'AJAP1',
 'NPHP4',
 'KCNAB2',
 'CHD5',
 'RPL22',
 'RNF207',
 'ICMT',
 'HES3',
 'GPR153',
 'ACOT7',
 'HES2',
 'ESPN',
 'TNFRSF25',
 'PLEKHG5',
 'NOL9',
 'TAS1R1',
 'ZBTB48',
 'KLH

In [7]:
adata.var['gene_name']=adata.var.index
adata.var['gene_name']

feature
TFAP2A      TFAP2A
DGCR2        DGCR2
ZNF703      ZNF703
MUL1          MUL1
TRMT61B    TRMT61B
            ...   
LAMTOR1    LAMTOR1
LRRC56      LRRC56
WFS1          WFS1
P2RX1        P2RX1
FOSB          FOSB
Name: gene_name, Length: 18752, dtype: category
Categories (59236, object): ['5S_rRNA_ENSG00000276861', '5S_rRNA_ENSG00000277411', '5S_rRNA_ENSG00000277488', '5S_rRNA_ENSG00000278457', ..., 'hsa-mir-1253', 'hsa-mir-423', 'hsa-mir-8069-1', 'snoZ196']

In [8]:
import numpy as np
# 初始化一个新的数据矩阵，形状为 (adata.X.shape[0], len(all_genes))，填充为零
new_data = np.zeros((adata.X.shape[0], len(all_genes)), dtype=np.float32)


In [9]:
existing_genes = adata.var['gene_name']
existing_genes

feature
TFAP2A      TFAP2A
DGCR2        DGCR2
ZNF703      ZNF703
MUL1          MUL1
TRMT61B    TRMT61B
            ...   
LAMTOR1    LAMTOR1
LRRC56      LRRC56
WFS1          WFS1
P2RX1        P2RX1
FOSB          FOSB
Name: gene_name, Length: 18752, dtype: category
Categories (59236, object): ['5S_rRNA_ENSG00000276861', '5S_rRNA_ENSG00000277411', '5S_rRNA_ENSG00000277488', '5S_rRNA_ENSG00000278457', ..., 'hsa-mir-1253', 'hsa-mir-423', 'hsa-mir-8069-1', 'snoZ196']

In [10]:
# 将所有基因名称转换为小写
all_genes_lower = [gene.lower() for gene in all_genes]
adata_genes_lower = [gene.lower() for gene in existing_genes]

# 将两个列表转换为集合
all_genes_set = set(all_genes_lower)
adata_genes_set = set(adata_genes_lower)

# 计算交集
matching_genes = all_genes_set.intersection(adata_genes_set)
matching_count = len(matching_genes)
# 计算不匹配的基因
non_matching_genes = adata_genes_set - matching_genes
non_matching_count = len(non_matching_genes)


# 输出结果
print(f"匹配的基因数量: {matching_count}")
print(f"匹配的基因列表: {matching_genes}")
non_matching_genes


匹配的基因数量: 18752
匹配的基因列表: {'pcdhb7', 'med30', 'rad51d', 'ifna4', 'c6orf15', 'dtwd2', 'ncoa1', 'hcrt', 'ccni2', 'fmn1', 'slc66a2', 'rnf44', 'trim34', 'prr20g', 'zmynd12', 'eif3m', 'slc25a13', 'gstt2b', 'ehmt2', 'rps6ka5', 'h3c10', 'rpl26l1', 'arrb2', 'kcnk16', 'grem1', 'tal1', 'spp2', 'ctbp1', 'rbbp8', 'arl13b', 'dpp4', 'osbpl8', 'npas1', 'mrpl45', 'arsk', 'mrpl17', 'cox7a2l', 'apln', 'eral1', 'ppp1r3g', 'znf525', 'gabrg3', 'ddx11', 'taf12', 'pabpc3', 'mctp1', 'nrm', 'cdk9', 'peg3', 'poteb3', 'napa', 'susd6', 'rif1', 'lbh', 'reep1', 'nup153', 'znf683', 'tslp', 'cpz', 'adgre3', 'mapk9', 'rnf113b', 'ugt2b17', 'snapc3', 'phf21a', 'best2', 'znf81', 'dennd2a', 'nop56', 'psg11', 'dph1', 'igsf9', 'pmch', 'or5m8', 'smim7', 'cyp1a2', 'celsr1', 'ubxn11', 'arhgef35', 'znf524', 'mrps9', 'zic1', 'tppp2', 'ccdc201', 'drp2', 'alg11', 'map3k14', 'sh2b3', 'adgrb1', 'tbc1d24', 'agtr1', 'swi5', 'c10orf95', 'abhd18', 'ssr2', 'scel', 'milr1', 'tnfrsf1a', 'ltc4s', 'slc9a8', 'smc5', 'gpr88', 'ptpn23', 'vegfc', 

set()

In [11]:
gene_to_index = {gene: idx for idx, gene in enumerate(all_genes_lower)}
gene_to_index

{'or4f5': 0,
 'or4f29': 1,
 'or4f16': 2,
 'samd11': 3,
 'noc2l': 4,
 'klhl17': 5,
 'plekhn1': 6,
 'perm1': 7,
 'hes4': 8,
 'isg15': 9,
 'agrn': 10,
 'rnf223': 11,
 'c1orf159': 12,
 'ttll10': 13,
 'tnfrsf18': 14,
 'tnfrsf4': 15,
 'sdf4': 16,
 'b3galt6': 17,
 'c1qtnf12': 18,
 'ube2j2': 19,
 'scnn1d': 20,
 'acap3': 21,
 'pusl1': 22,
 'ints11': 23,
 'cptp': 24,
 'tas1r3': 25,
 'dvl1': 26,
 'mxra8': 27,
 'aurkaip1': 28,
 'ccnl2': 29,
 'mrpl20': 30,
 'ankrd65': 31,
 'tmem88b': 32,
 'vwa1': 33,
 'atad3c': 34,
 'atad3b': 35,
 'atad3a': 36,
 'tmem240': 37,
 'ssu72': 38,
 'fndc10': 39,
 'mib2': 40,
 'mmp23b': 41,
 'cdk11b': 42,
 'slc35e2b': 43,
 'cdk11a': 44,
 'nadk': 45,
 'gnb1': 46,
 'calml6': 47,
 'tmem52': 48,
 'cfap74': 49,
 'gabrd': 50,
 'prkcz': 51,
 'faap20': 52,
 'ski': 53,
 'morn1': 54,
 'rer1': 55,
 'pex10': 56,
 'plch2': 57,
 'pank4': 58,
 'hes5': 59,
 'tnfrsf14': 60,
 'prxl2b': 61,
 'mmel1': 62,
 'ttc34': 63,
 'actrt2': 64,
 'prdm16': 65,
 'arhgef16': 66,
 'megf6': 67,
 'tprg1l': 68

In [12]:
only_in_all_genes = all_genes_set - adata_genes_set

only_in_adata_genes = adata_genes_set - all_genes_set

# 输出结果
print(f"仅在 all_genes 中存在的基因数量: {len(only_in_all_genes)}")
print(f"仅在 all_genes 中存在的基因: {only_in_all_genes}")

print(f"仅在 adata_genes 中存在的基因数量: {len(only_in_adata_genes)}")
print(f"仅在 adata_genes 中存在的基因: {only_in_adata_genes}")


仅在 all_genes 中存在的基因数量: 579
仅在 all_genes 中存在的基因: {'phb', 'lto1', 'gatd3a', 'fpgt-tnni3k', 'tp53tg3b', 'akr1c8p', 'cd99', 'krtap2-2', 'krtap10-10', 'pramef33', 'mtrnr2l3', 'tptep2-csnk1e', 'tbc1d3', 'bpy2c', 'npipa8', 'pmf1-bglap', 'abhd14a-acy1', 'ssu72p5', 'defb104b', 'akap17a', 'golga8o', 'urgcp-mrps24', 'trim49d2', 'znf781', 'opn1mw3', 'ino80b-wbp1', 'ct47a9', 'or11h12', 'defa3', 'ctage9', 'cfap298-tcp10l', 'usp17l5', 'micos10-nbl1', 'ppan-p2ry11', 'pramef14', 'specc1l-adora2a', 'usp17l25', 'ppial4e', 'pdzd3', 'trim51gp', 'arntl2', 'rfpl4al1', 'znf720', 'cgb3', 'c19orf54', 'hs3st3b1', 'zbed6', 'fam47e-stbd1', 'tcaf2c', 'tspy9p', 'znf722p', 'c11orf45', 'upk3bl1', 'usp17l29', 'tiaf1', 'bpy2b', 'npipb11', 'cenps-cort', 'bloc1s5-txndc5', 'ppial4c', 'vcy1b', 'h2ac14', 'ssu72p7', 'spdye15', 'c1qtnf3-amacr', 'rnf103-chmp3', 'frg2b', 'usp17l3', 'arl2-snx15', 'golga6l1', 'amy1c', 'or4f4', 'pdf', 'golga8s', 'cbwd1', 'kiaa0895l', 'uhrf1bp1', 'mtrnr2l6', 'amigo3', 'pramef26', 'h2bs1', 'or2a4', '

In [13]:
dense_adata_X = adata.X.toarray()
for i, gene in enumerate(adata_genes_lower):
    if gene in gene_to_index:
        new_data[:, gene_to_index[gene]] = dense_adata_X[:, i]
    else:
        print(f'Gene {gene} not found in all_genes list')

In [14]:
from sklearn.metrics import accuracy_score, f1_score
from sklearn.preprocessing import LabelEncoder
import numpy as np
from sklearn.model_selection import train_test_split


label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(adata.obs['cell_type'])  # 预先编码标签


random_seed = 42
X_train_val, X_test, y_train_val, y_test = train_test_split(
    new_data, labels_encoded, test_size=0.15, random_state=random_seed)


X_train, X_val, y_train, y_val = train_test_split(
    X_train_val, y_train_val, test_size=0.1765, random_state=random_seed)  # 0.1765 是为了让验证集占 15%

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
estim.model.to(device)
estim.model.eval()
with torch.no_grad():
    X_train_tensor = torch.tensor(X_train).float().to(device)
    X_test_tensor = torch.tensor(X_test).float().to(device)
    train_embeddings = estim.model.inner_model(X_train_tensor).detach().cpu().numpy()
    test_embeddings = estim.model.inner_model(X_test_tensor).detach().cpu().numpy()


cuda


In [15]:
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, f1_score, classification_report

    

    # 初始化和训练KNN分类器
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(train_embeddings, y_train)
    
    # 模型预测
predictions = knn.predict(test_embeddings)

    # 计算准确率和 F1 分数
accuracy = accuracy_score(y_test, predictions)
print(f"KNN Accuracy on Test Data: {accuracy}")
f1 = f1_score(y_test, predictions, average='weighted')
print(f"Weighted F1 Score: {f1}")
    
macro_f1 = f1_score(y_test, predictions, average='macro')
print(f'Macro F1 Score: {macro_f1}')

    # 计算随机猜测的准确率
class_probabilities = np.bincount(y_test) / len(y_test)
random_accuracy = np.sum(class_probabilities ** 2)
print(f"Random Guess Accuracy: {random_accuracy}")

    # 生成分类报告
report = classification_report(y_test, predictions, target_names=label_encoder.classes_)
print(report)

KNN Accuracy on Test Data: 0.9988174077578051
Weighted F1 Score: 0.9986830175367792
Macro F1 Score: 0.886906791168514
Random Guess Accuracy: 0.4721198844214674
                                        precision    recall  f1-score   support

                             astrocyte       1.00      1.00      1.00       567
     central nervous system macrophage       1.00      1.00      1.00       266
                      endothelial cell       1.00      1.00      1.00        28
                        ependymal cell       1.00      1.00      1.00        13
                            fibroblast       1.00      1.00      1.00        14
                             leukocyte       1.00      1.00      1.00         7
                                neuron       1.00      1.00      1.00      5486
                       oligodendrocyte       1.00      1.00      1.00      1786
        oligodendrocyte precursor cell       0.98      1.00      0.99       281
                              pericyte 

/home/hanchuangyi/miniconda3/envs/ssl/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/hanchuangyi/miniconda3/envs/ssl/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/hanchuangyi/miniconda3/envs/ssl/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.

In [16]:
with torch.no_grad():
    new_data_tensor = torch.tensor(new_data).float().to(device)
    SSL_embeddings = estim.model.inner_model(new_data_tensor).detach().cpu().numpy()
SSL_embeddings

array([[ -4.0070624 ,   7.225292  ,   9.554823  , ...,  -7.048564  ,
          7.511907  ,  -1.367258  ],
       [ -1.8960649 ,   4.972506  ,   6.817571  , ...,  -5.251363  ,
          8.816312  ,   2.0248954 ],
       [ -4.1966877 ,   6.1550913 ,   9.103974  , ...,  -6.2529974 ,
          6.3086557 ,  -1.3986075 ],
       ...,
       [ -5.2814245 ,   0.32106742,  11.365387  , ...,  -6.5598626 ,
          0.51578045,   1.0035208 ],
       [ -6.012898  ,   2.2098784 ,  13.160812  , ...,  -7.356     ,
          1.3691056 ,   0.12093989],
       [ -7.476328  ,  -1.154035  ,  16.567583  , ..., -15.65792   ,
         -1.7909765 ,  -0.90602565]], dtype=float32)

In [17]:

import pandas as pd
import os
import re

# 当前 Notebook 文件名
notebook_name = "HiT_CA4_DGC_barlow_twins_zero_shot_42.ipynb"

# 初始化需要打印的值
init_train_loss = train_losses[0] if 'train_losses' in globals() else None
init_val_loss = val_losses[0] if 'val_losses' in globals() else None
converged_epoch = len(train_losses) - patience if 'train_losses' in globals() else None
converged_val_loss = best_val_loss if 'best_val_loss' in globals() else None

# 打印所有所需的指标
print("Metrics Summary:")
if 'train_losses' in globals():
    print(f"init_train_loss\tinit_val_loss\tconverged_epoch\tconverged_val_loss\tmacro_f1\tweighted_f1\tmicro_f1")
    print(f"{init_train_loss:.3f}\t{init_val_loss:.3f}\t{converged_epoch}\t{converged_val_loss:.3f}\t{macro_f1:.3f}\t{f1:.3f}\t{accuracy:.3f}")
else:
    print(f"macro_f1\tweighted_f1\tmicor_f1")
    print(f"{macro_f1:.3f}\t{f1:.3f}\t{accuracy:.3f}")

# 保存结果到 CSV 文件
output_data = {
    'dataset_split_random_seed': [int(random_seed)],
    'dataset': ['HiT_CA4_DGC'],
    'method': [re.search(r'HiT_CA4_DGC_(.*?)_\d+', notebook_name).group(1)],
    'init_train_loss': [init_train_loss if init_train_loss is not None else ''],
    'init_val_loss': [init_val_loss if init_val_loss is not None else ''],
    'converged_epoch': [converged_epoch if converged_epoch is not None else ''],
    'converged_val_loss': [converged_val_loss if converged_val_loss is not None else ''],
    'macro_f1': [macro_f1],
    'weighted_f1': [f1],
    'micro_f1': [accuracy]
}
output_df = pd.DataFrame(output_data)

# 保存到当前目录下名为 results 的文件夹中
if not os.path.exists('results'):
    os.makedirs('results')

csv_filename = f"results/{os.path.splitext(notebook_name)[0]}_results.csv"
output_df.to_csv(csv_filename, index=False)


Metrics Summary:
macro_f1	weighted_f1	micor_f1
0.887	0.999	0.999
